![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_BERT_TOKEN_CLASSIFIER.ipynb)

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## 1. Colab Setup

Import license keys

In [ ]:
%%capture
import os
import json

with open(r'/content/spark_nlp_for_healthcare.json') as f:
    license_keys = json.load(f)
    
locals().update(license_keys)

for k,v in license_keys.items(): 
    %set_env $k=$v

In [ ]:
license_keys['PUBLIC_VERSION']

'3.2.3'

In [ ]:
%%capture

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

In [ ]:
import os 

os.environ["SPARK_NLP_LICENSE"] = SPARK_NLP_LICENSE    
os.environ["SECRET"] = SECRET
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID     
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY

In [ ]:
%%capture
! pip install --upgrade spark-nlp==$PUBLIC_VERSION findspark
! pip install --upgrade spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET
! pip install --ignore-installed spark-nlp-display

In [ ]:
# Install java
! sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)


In [ ]:
import sparknlp

print (f"sparknlp version: {sparknlp.version()}")
import pandas as pd
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *

from sparknlp.base import *
import pyspark.sql.functions as F

# import internal package
import sparknlp_jsl
from sparknlp_jsl.annotator import *

print (f"sparknlp-jsl version: {sparknlp.version()}")

spark = sparknlp_jsl.start(license_keys['SECRET'])

sparknlp version: 3.3.1
sparknlp-jsl version: 3.3.1


## 2. Create example inputs

In [ ]:
text_list_jsl = [
             """The patient is a 30-year-old female with a long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. She did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with her previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, Xenaderm to wounds topically b.i.d., Lantus 40 units subcutaneously at bedtime, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. daily, Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., Percocet 5/325 mg 2 tablets q.4 h. p.r.n., magnesium citrate 1 bottle p.o. p.r.n., sliding scale coverage insulin, Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d."""]

In [ ]:
text_list_drug = [
    "The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomic organization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene for type II diabetes mellitus in the Pima Indian population. The gene spans approximately 7.6 kb and contains one noncoding and two coding exons separated by approximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Our expression studies revealed the presence of the transcript in various human tissues including the pancreas, and two major insulin-responsive tissues: fat and skeletal muscle. The characterization of the KCNJ9 gene should facilitate further studies on the function of the KCNJ9 protein and allow evaluation of the potential role of the locus in Type II diabetes.BACKGROUND: At present, it is one of the most important issues for the treatment of breast cancer to develop the standard therapy for patients previously treated with anthracyclines and taxanes. With the objective of determining the usefulness of vinorelbine monotherapy in patients with advanced or recurrent breast cancer after standard therapy, we evaluated the efficacy and safety of vinorelbine in patients previously treated with anthracyclines and taxanes."]

In [ ]:
text_list_deid = [
    """HISTORY OF PRESENT ILLNESS: Mr. Smith is a 60-year-old white male veteran with multiple comorbidities, who has a history of bladder cancer diagnosed approximately two years ago by the VA Hospital. He underwent a resection there. He was to be admitted to the Day Hospital for cystectomy. He was seen in Urology Clinic and Radiology Clinic on 02/04/2003.

HOSPITAL COURSE: Mr. Smith presented to the Day Hospital in anticipation for Urology surgery. On evaluation, EKG, echocardiogram was abnormal, a Cardiology consult was obtained. A cardiac adenosine stress MRI was then proceeded, same was positive for inducible ischemia, mild-to-moderate inferolateral subendocardial infarction with peri-infarct ischemia. In addition, inducible ischemia seen in the inferior lateral septum. Mr. Smith underwent a left heart catheterization, which revealed two vessel coronary artery disease. The RCA, proximal was 95% stenosed and the distal 80% stenosed. The mid LAD was 85% stenosed and the distal LAD was 85% stenosed. There was four Multi-Link Vision bare metal stents placed to decrease all four lesions to 0%. Following intervention, Mr. Smith was admitted to 7 Ardmore Tower under Cardiology Service under the direction of Dr. Hart. Mr. Smith had a noncomplicated post-intervention hospital course. He was stable for discharge home on 02/07/2003 with instructions to take Plavix daily for one month and Urology is aware of the same."""]

In [ ]:
text_list_jsl_slim = ["""HPI: A 69-year-old white female with a history of metastatic breast cancer, depression, anxiety, recent UTI, and obstructive uropathy, admitted to the ABCD Hospital on February 6, 2007, for lightheadedness, weakness, and shortness of breath. The patient was consulted by Psychiatry for anxiety. I know this patient from a previous consult. During this recent admission, the patient has experienced anxiety and had a panic attack yesterday with "syncopal episodes." She was given Ativan 0.25 mg on a p.r.n. basis with relief after one to two hours. The patient was seen by Abc, MD, and Def, Ph.D. The laboratories were reviewed and were positive for UTI, and anemia is also present. The TSH level was within normal limits. She previously responded well to trazodone for depression, poor appetite, and decreased sleep and anxiety. A low dose of Klonopin was also helpful for sedation.

PAST MEDICAL HISTORY: Metastatic breast cancer to bone. The patient also has a history of hypertension, hypothyroidism, recurrent UTI secondary to obstruction of left ureteropelvic junction, cholelithiasis, chronic renal insufficiency, Port-A-Cath placement, and hydronephrosis.

PAST PSYCHIATRIC HISTORY: The patient has a history of depression and anxiety. She was taking Remeron 15 mg q.h.s., Ambien 5 mg q.h.s. on a p.r.n. basis, Ativan 0.25 mg every 6 hours on a p.r.n. basis, and Klonopin 0.25 mg at night while she was at home.""",
 """The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioning yellow discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but negative for any perioral cyanosis or retractions). One day ago, mom also noticed a tactile temperature and gave the patient Tylenol. Baby also has had some decreased p.o. intake. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired and has been fussy over the past 2 days. The parents noticed no improvement with albuterol treatments given in the ER. His urine output has also decreased; normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His bowel movements are yellow colored and soft in nature.""",
 """ The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. HCTZ 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.""",
 """ HISTORY OF PRESENT ILLNESS: The patient is a 68-year-old Korean gentleman with a history of coronary artery disease, hypertension, diabetes and stage III CKD with a creatinine of 1.8 in May 2006 corresponding with the GFR of 40-41 mL/min. The patient had blood work done at Dr. XYZ's office on June 01, 2006, which revealed an elevation in his creatinine up to 2.3. He was asked to come in to see a nephrologist for further evaluation. I am therefore asked by Dr. XYZ to see this patient in consultation for evaluation of acute on chronic kidney failure. The patient states that he was actually taking up to 12 to 13 pills of Chinese herbs and dietary supplements for the past year. He only stopped about two or three weeks ago. He also states that TriCor was added about one or two months ago but he is not sure of the date. He has not had an ultrasound but has been diagnosed with prostatic hypertrophy by his primary care doctor and placed on Flomax. He states that his urinary dribbling and weak stream had not improved since doing this. For the past couple of weeks, he has had dizziness in the morning. This is then associated with low glucose. However the patient's blood glucose this morning was 123 and he still was dizzy. This was worse on standing. He states that he has been checking his blood pressure regularly at home because he has felt so bad and that he has gotten under 100/60 on several occasions. His pulses remained in the 60s.

ALLERGIES: None.

MEDICATIONS: Imdur 20 mg two to three times daily, nitroglycerin p.r.n., insulin 70/30 40/45 units daily, Zetia 10 mg daily, ? Triglide 50 mg daily, Prevacid 30 mg daily, Plavix 75 mg daily, potassium 10 mEq daily, Lasix 60 mg daily, folate 1 mg b.i.d., Niaspan 500 mg daily, atenolol 50 mg daily, enalapril 10 mg b.i.d., glyburide 10 mg b.i.d., Xanax 0.25 mg b.i.d., aspirin 325 mg daily, Tylenol p.r.n., Zantac 150 mg b.i.d., Crestor 5 mg daily, TriCor 145 mg daily, Digitek 0.125 mg daily, Celexa 20 mg daily, and Flomax 0.4 mg daily.""",
 """Mr. ABC is a 60-year-old gentleman who had a markedly abnormal stress test earlier today in my office with severe chest pain after 5 minutes of exercise on the standard Bruce with horizontal ST depressions and moderate apical ischemia on stress imaging only. He required 3 sublingual nitroglycerin in total (please see also admission history and physical for full details). 

The patient underwent cardiac catheterization with myself today which showed mild-to-moderate left main distal disease of 30%, moderate proximal LAD with a severe mid-LAD lesion of 99%, and a mid-left circumflex lesion of 80% with normal LV function and some mild luminal irregularities in the right coronary artery with some moderate stenosis seen in the mid to distal right PDA.

I discussed these results with the patient, and he had been relating to me that he was having rest anginal symptoms, as well as nocturnal anginal symptoms, and especially given the severity of the mid left anterior descending lesion, with a markedly abnormal stress test, I felt he was best suited for transfer for PCI. I discussed the case with Dr. X at Medical Center who has kindly accepted the patient in transfer.

CONDITION ON TRANSFER: Stable but guarded. The patient is pain-free at this time.

MEDICATIONS ON TRANSFER:
1. Aspirin 325 mg once a day.
2. Metoprolol 50 mg once a day, but we have had to hold it because of relative bradycardia which he apparently has a history of.
3. Nexium 40 mg once a day.
4. Zocor 40 mg once a day, and there is a fasting lipid profile pending at the time of this dictation. I see that his LDL was 136 on May 3, 2002.
5. Plavix 600 mg p.o. x1 which I am giving him tonight."""]

In [ ]:
text_list_bacteria = ["""Based on these genetic and phenotypic properties, we propose that strain SMSP (T) represents \
a novel species of the genus Methanoregula, for which we propose the name Methanoregula formicica \
sp. nov., with the type strain SMSP (T) (= NBRC 105244 (T) = DSM 22288 (T))."""]

In [ ]:
text_list_ade = ["""Been taking Lipitor for 15 years , have experienced severe fatigue a lot!!! . Doctor moved me to voltaren 2 months ago , so far , have only experienced cramps"""]

In [ ]:
text_list_anatomy = ["""This is an 11-year-old female who comes in for two different things. 1. She was seen by the allergist. No allergies present, so she stopped her Allegra, but she is still real congested and does a lot of snorting. They do not notice a lot of snoring at night though, but she seems to be always like that. 2. On her right great toe, she has got some redness and erythema. Her skin is kind of peeling a little bit, but it has been like that for about a week and a half now.\nGeneral: Well-developed female, in no acute distress, afebrile.\nHEENT: Sclerae and conjunctivae clear. Extraocular muscles intact. TMs clear. Nares patent. A little bit of swelling of the turbinates on the left. Oropharynx is essentially clear. Mucous membranes are moist.\nNeck: No lymphadenopathy.\nChest: Clear.\nAbdomen: Positive bowel sounds and soft.\nDermatologic: She has got redness along her right great toe, but no bleeding or oozing. Some dryness of her skin. Her toenails themselves are very short and even on her left foot and her left great toe the toenails are very short."""]

In [ ]:
text_list_chemicals = ["""The results have shown that the product p - choloroaniline is not a significant factor in chlorhexidine - digluconate associated erosive cystitis. "A high percentage of kanamycin - colistin and povidone - iodine irrigations were associated with erosive cystitis."""]

In [ ]:
text_list_chemprot = ["Keratinocyte growth factor and acidic fibroblast growth factor are mitogens for primary cultures of mammary epithelium."]

In [ ]:
text_list_ner_bionlp = ["""The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes."""]

## 3. Select the NER model, construct the pipeline and visualize the results.

Select the NER model - Models: **'bert_token_classifier_ner_jsl', 'bert_token_classifier_ner_drugs', 'bert_token_classifier_ner_deid', 'bert_token_classifier_ner_bacteria','bert_token_classifier_ner_ade','bert_token_classifier_ner_anatomy', 'bert_token_classifier_ner_chemicals','bert_token_classifier_ner_chemprot','bert_token_classifier_ner_bionlp'**

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [ ]:
model_list = ['bert_token_classifier_ner_jsl','bert_token_classifier_ner_drugs','bert_token_classifier_ner_deid','bert_token_classifier_ner_bacteria','bert_token_classifier_ner_ade','bert_token_classifier_ner_anatomy', 'bert_token_classifier_ner_chemicals','bert_token_classifier_ner_chemprot', 'bert_token_classifier_ner_bionlp']


In [ ]:
from sparknlp_display import NerVisualizer
for MODEL_NAME in model_list:

    documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

    sentenceDetector = SentenceDetectorDLModel.pretrained() \
          .setInputCols(["document"]) \
          .setOutputCol("sentence") 

    tokenizer = Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")
      
    tokenClassifier = BertForTokenClassification.pretrained( MODEL_NAME, "en", 'clinical/models')\
      .setInputCols("sentence","token")\
      .setOutputCol("ner")\
      .setCaseSensitive(True)

    ner_converter = NerConverter()\
      .setInputCols(["sentence","token","ner"])\
      .setOutputCol("ner_chunk")

    pipeline =  Pipeline(stages=[documentAssembler,sentenceDetector, tokenizer, tokenClassifier, ner_converter])
    pipelineModel = pipeline.fit(spark.createDataFrame([['']]).toDF("text"))
    
    if MODEL_NAME == "bert_token_classifier_ner_jsl":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_jsl}))
    elif MODEL_NAME == "bert_token_classifier_ner_drugs":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_drug}))
    elif MODEL_NAME == "bert_token_classifier_ner_deid":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_deid}))
    elif MODEL_NAME == "bert_token_classififer_ner_bacteria":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_bacteria}))
    elif MODEL_NAME == "bert_token_classifier_ner_jsl_slim":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_jsl_slim}))
    elif MODEL_NAME == "bert_token_classifier_ner_ade":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_ade}))
    elif MODEL_NAME == "bert_token_classifier_ner_anatomy":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_anatomy}))
    elif MODEL_NAME == "bert_token_classifier_ner_chemicals":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_chemicals}))
    elif MODEL_NAME == "bert_token_classifier_ner_bionlp":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_ner_bionlp}))
    elif MODEL_NAME == "bert_token_classifier_ner_chemprot":
      df = spark.createDataFrame(pd.DataFrame({"text": text_list_chemprot}))


    print("<----------------- MODEL NAME:","\033[1m" + MODEL_NAME + "\033[0m"," ----------------- >")
    result = pipelineModel.transform(df)
    result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
          .select(F.expr("cols['0']").alias("chunk"),
                  F.expr("cols['1']['entity']").alias("ner_label"))\
          .show(truncate=False)

    NerVisualizer().display(
        result = result.collect()[0],
        label_col = 'ner_chunk',
        document_col = 'document'
    )

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_jsl download started this may take some time.
Approximate size to download 385.8 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_jsl  ----------------- >
+---------------------------+----------------------------+
|chunk                      |ner_label                   |
+---------------------------+----------------------------+
|30-year-old                |Age                         |
|female                     |Gender                      |
|insulin dependent          |Diabetes                    |
|diabetes                   |Diabetes                    |
|type 2                     |Diabetes                    |
|coronary artery disease    |Heart_Disease               |
|chronic renal insufficiency|Kidney_Disease              |
|peripheral vascular disease|Disease_Syndrome_Disorder   |
|diabetes                   |Diabetes         

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_drugs download started this may take some time.
Approximate size to download 385.3 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_drugs  ----------------- >
+--------------+---------+
|chunk         |ner_label|
+--------------+---------+
|potassium     |DrugChem |
|nucleotide    |DrugChem |
|anthracyclines|DrugChem |
|taxanes       |DrugChem |
|vinorelbine   |DrugChem |
|vinorelbine   |DrugChem |
|anthracyclines|DrugChem |
|taxanes       |DrugChem |
+--------------+---------+



sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ner_deid download started this may take some time.
Approximate size to download 385.4 MB
[OK!]
<----------------- MODEL NAME: bert_token_classifier_ner_deid  ----------------- >
+---------------+---------+
|chunk          |ner_label|
+---------------+---------+
|Smith          |PATIENT  |
|60-year-old    |AGE      |
|VA Hospital    |HOSPITAL |
|Day Hospital   |HOSPITAL |
|02/04/2003     |DATE     |
|Smith          |PATIENT  |
|Day Hospital   |HOSPITAL |
|Smith          |PATIENT  |
|Smith          |PATIENT  |
|7 Ardmore Tower|STREET   |
|Hart           |DOCTOR   |
|Smith          |PATIENT  |
|02/07/2003     |DATE     |
+---------------+---------+

